# Model Risk Pipeline

_Initial commit: Anton Markov, 19 November 2021_

Основная цель данного ноутбука — построить базовую структуру пайплайна с учетом оптимизации гиперпараметров.

Реализована оптимизация через `hyperopt`, в будущем возможно поддержка иных библиотек.

__Входные данные:__

1. Датасет
2. Модель
3. Список модулей, которые могут оптимизироваться в качества гиперпараметра

__Исходящие данные:__

1. Оптимальный набор модулей, согласно `hyperopt`
2. Параметры обученной оптимальной модели


## 1. Technicals

In [1]:
import time
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets, metrics, model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from hyperopt import hp
# from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# for HyperOpt class
import lightgbm as lgb
import xgboost as xgb
# import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

In [2]:
# новый пакет!
from feature_engine.encoding import WoEEncoder
from feature_engine.creation import CombineWithReferenceFeature
from feature_engine.selection import RecursiveFeatureAddition
from feature_engine.encoding import OneHotEncoder

In [3]:
from sklearn.pipeline import Pipeline
import sklearn
import umap

In [4]:
with open('../datasets/01_german/factors.json') as json_file:
    factors_dict = json.load(json_file)

In [5]:
factors_dict['cat_vals']

['cheq_acc',
 'cred_hist',
 'purp',
 'save_acc',
 'empl_t',
 'pers_status',
 'guarant_flg',
 'prop',
 'inst_plan',
 'house',
 'job',
 'tel_flg',
 'foreign_flg']

In [6]:
seed = 42

In [7]:
def Gini(y, y_pred):
    res = roc_auc_score(y, y_pred) * 2 - 1
    print(f"Gini: {res}")
    return(res)

### Datasets

In [8]:
X_train = pd.read_parquet('../datasets/01_german/samples/X_train.parquet')
y_train = pd.read_parquet('../datasets/01_german/samples/y_train.parquet').target

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_test  = pd.read_parquet('../datasets/01_german/samples/X_test.parquet')
y_test  = pd.read_parquet('../datasets/01_german/samples/y_test.parquet').target

### Модули

Для примера использую облегченную версию пайплайна:
    
0. Входные данные для обучения - обучающая и валидационная выборки,
1. Энкодинг категориальных переменных:
    + OneHotEncoder
    + WOE
3. Feature Engineering:
    + PCA
    + Kernel PCA
    + CombineWithReferenceFeature
    + _отсутствует_
4. Feature Selection:
    + RecursiveFeatureAddition
    + _отсутствует_
5. LightGBM


Все параметры модулей оптимизируются. Кроме того, оптимизируется сам набор модулей.

In [9]:
class CombineWithReferenceFeature_adj():
    '''
    Обертка вокруг CombineWithReferenceFeature()
    Позволяет не устанавливать параметры
    + variables_to_combine
    + reference_variables
    заранее (иначе не будет работать с OneHotEncoder
    и прочими преобразователями данных, а делать это при .fit()
    '''
    def __init__(self, operations):
        self.operations = operations
        
    def fit(self, X, y):
        self.combinator = CombineWithReferenceFeature(
            variables_to_combine = list(X.columns),
            reference_variables = list(X.columns),
            operations = self.operations
        )
        self.combinator.fit(X, y)
        return(self)
    
    def transform(self, X):
        return(self.combinator.transform(X))        

In [10]:
class KernelPCA_adj():
    '''
    Обертка нужна, чтобы transform() возвращал
    pd.df(), а не np.array() - не все могут в np
    '''
    def __init__(self, **kwargs):
        self.pca = sklearn.decomposition.KernelPCA(**kwargs)
        
    def fit(self, X, y):
        self.pca.fit(X, y)
        return self
    
    def transform(self, X):
        # potentially 
        return pd.DataFrame(self.pca.transform(X), index = X.index)
    
    def set_params(self, **kwargs):
        self.pca.set_params(**kwargs)
        return self 

In [11]:
# from umap import UMAP

In [12]:
woe = WoEEncoder(variables = factors_dict['cat_vals'])
onehot = OneHotEncoder(variables = factors_dict['cat_vals'])

kPCA = KernelPCA_adj(
    n_components = 8,  # сколько оставить компонентов; по дефолту - все
    kernel = "linear", # ядро. По дфеолту линейное. Можно сделать своё, но тогда его нужно предварительно вычислить отдельно,
                       # поставить kernel = "precomputed" и передать уже вычисленное ядро в качестве X
    degree = 3,        # степень полинома для некоторых типов ядер. Важный параметр для тьюнинга, но сильно напрягает процессор
    n_jobs = -1        # объект умеет быть многопоточным! -1 займет все ядра
)

feat_eng = CombineWithReferenceFeature_adj(
    operations = ['mul']
)

lgbm_mdl = LGBMClassifier(
    num_leaves = 10,
    learning_rate = .1,
    reg_alpha = 8,
    reg_lambda = 8,
    random_state = seed
)

feat_sel = RecursiveFeatureAddition(
    lgbm_mdl,
    threshold = 0.005
)

### Проверка работы Pipeline

In [13]:
mdl_pipe = Pipeline([
    # ('woe', woe),
    ('onehot', onehot), # must-have
    ('kPCA', kPCA),
    # ('UMAPer', UMAPer),
    ('feat_eng', feat_eng),
    # ('feat_select', feat_sel),
    ('lgbm', lgbm_mdl) # must-have
])

In [14]:
mdl_pipe.fit(X_train, y_train)
Gini(y_train, mdl_pipe.predict_proba(X_train)[:, 1])
Gini(y_test, mdl_pipe.predict_proba(X_test)[:, 1])

Gini: 0.8604799619949672
Gini: 0.6304844332251474


0.6304844332251474

## 2. Оптимизация параметров в конкретном пайплайне

Хорошая статья с примером [здесь](https://towardsdatascience.com/an-example-of-hyperparameter-optimization-on-xgboost-lightgbm-and-catboost-using-hyperopt-12bc41a271e).

Ещё раз приведём все модули, которые используются для примера:
    
0. Входные данные для обучения - обучающая и валидационная выборки,
1. Энкодинг категориальных переменных:
    + OneHotEncoder
    + WOE
3. Feature Engineering:
    + PCA
    + Kernel PCA
    + CombineWithReferenceFeature
    + _отсутствует_
4. Feature Selection:
    + RecursiveFeatureAddition
    + _отсутствует_
5. LightGBM

Все параметры модулей оптимизируются. Кроме того, оптимизируется сам набор модулей.

Now let's put `hyperopt` on top of that:

Optimizer definition:

In [15]:
set_params = {
    'kPCA__n_components':     hp.choice('kPCA__n_components',     np.arange(5, 11)),
    'lgbm__learning_rate':    hp.choice('lgbm__learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'lgbm__num_leaves':       hp.choice('lgbm__num_leaves',       np.arange(5, 16, 1, dtype=int)),
    'lgbm__reg_alpha':        hp.choice('lgbm__reg_alpha',        np.arange(0, 16, 1, dtype=int)),
    'lgbm__reg_lambda':       hp.choice('lgbm__reg_lambda',       np.arange(0, 16, 1, dtype=int)),
    'lgbm__n_estimators':     100,
}

pipe_para = dict()
pipe_para['set_params']     = set_params
pipe_para['loss_func']      = lambda y, pred: -sklearn.metrics.roc_auc_score(y, pred)
# pipe_para['loss_func']      = lambda y, pred: -sklearn.metrics.log_loss(y, pred)

In [16]:
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

class PipeHPOpt(object):

    def __init__(self, X, y, reg, mode='kfold', n_folds = 5, test_size=.33, seed=42):
        
        if (mode != 'kfold') & (mode != 'valid'):
            raise ValueError("Choose mode 'kfold' or 'valid'")
        if (mode == 'valid') & (n_folds != 5):
            import warnings
            warnings.warn("Non-default n_folds won't be used since mode == valid!")
        if (mode == 'kfold') & (test_size != .33):
            import warnings
            warnings.warn("Non-default test_size won't be used since mode == kfold!")
            
        self.X       = X
        self.y       = y
        self.mode    = mode
        self.n_folds = n_folds
        self.seed    = seed
        self.reg     = reg 
        
        if mode == 'valid':
            self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
                X, y, test_size=test_size, random_state=seed
            )

    def process(self, space, trials, algo, max_evals, fn_name='pipe'):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def pipe(self, para):
        reg = self.reg
        reg.set_params(**para['set_params'])
        if self.mode == 'kfold':
            return self.train_reg_kfold(reg, para)
        elif self.mode == 'valid':
            return self.train_reg_valid(reg, para)

    def train_reg_valid(self, reg, para):
        reg.fit(self.x_train, self.y_train)
        pred = reg.predict_proba(self.x_test)[:, 1]
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'model': reg, 'params': para['set_params'], 'status': STATUS_OK}
    
    def train_reg_kfold(self, reg, para):
        kf = KFold(n_splits=5, shuffle=True, random_state=self.seed)
        losses = []
        for train_index, test_index in kf.split(self.X):
            X_split_train, X_split_test = self.X.iloc[train_index, :], self.X.iloc[test_index, :]
            y_split_train, y_split_test = self.y.iloc[train_index, ],  self.y.iloc[test_index, ]
            reg.fit(X_split_train, y_split_train)
            pred = reg.predict_proba(X_split_test)[:, 1]
            loss = para['loss_func'](y_split_test, pred)
            losses.append(loss)
        return {'loss': np.mean(losses), 'params': para['set_params'], 'status': STATUS_OK}

In [17]:
modules = {
    'woe':         woe,
    'onehot':      onehot,
    'kPCA':        kPCA,
    'feat_eng':    feat_eng,
    'feat_sel':    feat_sel,
    'lgbm':        lgbm_mdl
}

In [18]:
reg = Pipeline([
    # ('woe',       self.mudules['woe']),
    ('onehot',      modules['onehot']), # must-have
    ('kPCA',        modules['kPCA']),
    # ('UMAPer',    self.mudules['UMAPer']),
    ('feat_eng',    modules['feat_eng']),
    # ('feat_select', self.mudules['feat_sel']),
    ('lgbm',        modules['lgbm']) # must-have
])

In [19]:
hpoptimizer = PipeHPOpt(X_train, y_train, reg=reg, mode='kfold', n_folds = 5, seed=seed)
lgb_opt, trials = hpoptimizer.process(space=pipe_para, trials=Trials(), algo=tpe.suggest, max_evals=10)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:33<00:00,  3.32s/trial, best loss: -0.7383439048960536]


In [20]:
# Best model нельзя вытащить при обучении k_fold, требуется только переобучение с данными гиперпараметрами
best_params = trials.results[np.argmin([r['loss'] for r in 
    trials.results])]['params']
worst_params = trials.results[np.argmax([r['loss'] for r in 
    trials.results])]['params']

## 3. Оптимизация структуры пайплайна

### utils

In [21]:
def filter_params(params, pipe):
    '''
    From all input parameters filter only
    those that are relevant for the current
    pipeline
    '''
    pipe_steps = list(pipe.named_steps.keys())
    params_keys = list(params.keys())
    
    return {
        key: params[key]
        for key in params_keys
        if key.split('__')[0] in pipe_steps
    }

In [22]:
## test

# pipe = Pipeline([
#     # ('woe',       self.mudules['woe']),
#     ('onehot',      modules['onehot']), # must-have
#     ('kPCA',        modules['kPCA']),
#     # ('UMAPer',    self.mudules['UMAPer']),
#     ('feat_eng',    modules['feat_eng']),
#     # ('feat_select', self.mudules['feat_sel']),
#     # ('lgbm',        modules['lgbm']) # must-have
# ])

# params = {
#     'kPCA__n_components':     8,
#     'lgbm__learning_rate':    .1,
#     'lgbm__num_leaves':       10,
#     'lgbm__reg_alpha':        8,
#     'lgbm__reg_lambda':       8,
#     'lgbm__n_estimators':     100
# }

# filter_params(params, pipe)

In [23]:
def construct_pipe(steps_dict, modules):
    '''
    Construct a pipeline given structure
    '''
    return [(steps_dict[s], modules[steps_dict[s]]) for s in steps_dict if steps_dict[s] != 'skip']

In [24]:
## test
# construct_pipe({'cat_encoding': 'onehot', 'feat_eng': 'kPCA'}, modules)

### optimizer

При добавлении модуля нужно включить его в словарь.

In [25]:
modules = {
    'woe':         woe,
    'onehot':      onehot,
    'kPCA':        kPCA,
    'feat_eng':    feat_eng,
    'feat_sel':    feat_sel,
    'lgbm':        lgbm_mdl
}

Сюда добавляем описание того, как новый модуль будет использоваться в структуре пайплайна:

In [26]:
pipe_params = {
    'cat_encoding':           hp.choice('cat_encoding', ['onehot']), # , 'woe' пропустить нельзя из-за наличия кат. пер-х
    'feat_eng':               hp.choice('feat_eng',     ['skip', 'kPCA']),
    'feat_sel':               hp.choice('feat_sel',     ['skip', 'feat_sel']),
    'lgbm':                   'lgbm'
}

Сюда добавляем гиерпараметры новго модуля:

In [27]:
set_params = {
    'kPCA__n_components':     hp.choice('kPCA__n_components',     np.arange(5, 11)),
    'lgbm__learning_rate':    hp.choice('lgbm__learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'lgbm__num_leaves':       hp.choice('lgbm__num_leaves',       np.arange(5, 16, 1, dtype=int)),
    'lgbm__reg_alpha':        hp.choice('lgbm__reg_alpha',        np.arange(0, 16, 1, dtype=int)),
    'lgbm__reg_lambda':       hp.choice('lgbm__reg_lambda',       np.arange(0, 16, 1, dtype=int)),
    'lgbm__n_estimators':     100,
}

In [28]:
pipe_para = dict()
pipe_para['pipe_params']    = pipe_params
pipe_para['set_params']     = set_params
pipe_para['loss_func']      = lambda y, pred: -sklearn.metrics.roc_auc_score(y, pred)
# pipe_para['loss_func']      = lambda y, pred: -sklearn.metrics.log_loss(y, pred)

In [29]:
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

class PipeHPOpt(object):

    def __init__(self, X, y, modules, mode='kfold', n_folds = 5, test_size=.33, seed=42):
        
        if (mode != 'kfold') & (mode != 'valid'):
            raise ValueError("Choose mode 'kfold' or 'valid'")
        if (mode == 'valid') & (n_folds != 5):
            import warnings
            warnings.warn("Non-default n_folds won't be used since mode == valid!")
        if (mode == 'kfold') & (test_size != .33):
            import warnings
            warnings.warn("Non-default test_size won't be used since mode == kfold!")
            
        self.X       = X
        self.y       = y
        self.mode    = mode
        self.n_folds = n_folds
        self.seed    = seed
        self.reg     = reg 
        self.modules = modules
        
        if mode == 'valid':
            self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
                X, y, test_size=test_size, random_state=seed
            )

    def process(self, space, trials, algo, max_evals, fn_name='pipe'):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def pipe(self, para):
        # print(para)
        pipe_steps = [(para['pipe_params'][i], modules[para['pipe_params'][i]]) for i in para['pipe_params'] if para['pipe_params'][i] != 'skip']
        reg = Pipeline(pipe_steps)
        for p in pipe_para['set_params']:
            try:
                reg.set_params({p: para[p]})
            except:
                pass
        if self.mode == 'kfold':
            return self.train_reg_kfold(reg, para)
        elif self.mode == 'valid':
            return self.train_reg_valid(reg, para)

    def train_reg_valid(self, reg, para):
        reg.fit(self.x_train, self.y_train)
        pred = reg.predict_proba(self.x_test)[:, 1]
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'model': reg, 'params': para, 'status': STATUS_OK}
    
    def train_reg_kfold(self, reg, para):
        kf = KFold(n_splits=5, shuffle=True, random_state=self.seed)
        losses = []
        for train_index, test_index in kf.split(self.X):
            X_split_train, X_split_test = self.X.iloc[train_index, :], self.X.iloc[test_index, :]
            y_split_train, y_split_test = self.y.iloc[train_index, ],  self.y.iloc[test_index, ]
            reg.fit(X_split_train, y_split_train)
            pred = reg.predict_proba(X_split_test)[:, 1]
            loss = para['loss_func'](y_split_test, pred)
            losses.append(loss)
        return {'loss': np.mean(losses), 'params': para, 'status': STATUS_OK}

In [30]:
hpoptimizer = PipeHPOpt(X_train, y_train, modules=modules, mode='kfold', n_folds = 5, seed=seed)
lgb_opt, trials = hpoptimizer.process(space=pipe_para, trials=Trials(), algo=tpe.suggest, max_evals=1000)

# hpoptimizer = PipeStructHPOpt(X_train, y_train, modules, space_params=set_params, mode='kfold', n_folds = 5, seed=seed)
# lgb_opt, trials = hpoptimizer.process(space_steps=steps, trials=Trials(), algo=tpe.suggest, max_evals=10)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:26:10<00:00,  5.17s/trial, best loss: -0.7731525296887065]


In [31]:
lgb_opt

{'cat_encoding': 0,
 'feat_eng': 0,
 'feat_sel': 0,
 'kPCA__n_components': 1,
 'lgbm__learning_rate': 4,
 'lgbm__num_leaves': 4,
 'lgbm__reg_alpha': 14,
 'lgbm__reg_lambda': 12}

In [32]:
best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']
best_params

{'loss_func': <function __main__.<lambda>(y, pred)>,
 'pipe_params': {'cat_encoding': 'onehot',
  'feat_eng': 'skip',
  'feat_sel': 'skip',
  'lgbm': 'lgbm'},
 'set_params': {'kPCA__n_components': 6,
  'lgbm__learning_rate': 0.25,
  'lgbm__n_estimators': 100,
  'lgbm__num_leaves': 9,
  'lgbm__reg_alpha': 14,
  'lgbm__reg_lambda': 12}}

# Archive code

In [33]:
# para = {
#     'loss_func': lambda y, pred: -sklearn.metrics.roc_auc_score(y, pred), 
#     'pipe_params': {'cat_encoding': 'onehot', 'feat_eng': 'skip'}, 
#     'set_params': {
#         'kPCA__n_components': 10, 
#         'lgbm__learning_rate': 0.2, 
#         'lgbm__n_estimators': 100, 
#         'lgbm__num_leaves': 9, 
#         'lgbm__reg_alpha': 4, 
#         'lgbm__reg_lambda': 3
#     }
# }

In [34]:
# for i in para['pipe_params']:
#     if i != 'skip':
#         print(para['pipe_params'][i])

In [35]:
# print(para)
# pipe_steps = [(para['pipe_params'][i], modules[para['pipe_params'][i]]) for i in para['pipe_params'] if para['pipe_params'][i] != 'skip']
# reg = Pipeline(pipe_steps)
# for p in pipe_para['set_params']:
#     try:
#         reg.set_params({p: para[p]})
#     except:
#         pass
# if self.mode == 'kfold':
#     return self.train_reg_kfold(reg, para)
# elif self.mode == 'valid':
#     return self.train_reg_valid(reg, para)

In [36]:
# # import lightgbm as lgb
# # import xgboost as xgb
# # import catboost as ctb
# from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
# from sklearn.model_selection import KFold
# from sklearn.model_selection import train_test_split

# class PipeStructHPOpt(object):

#     def __init__(self, X, y, modules, space_params, mode='kfold', n_folds = 5, test_size=.33, seed=42):
            
#         self.X            = X
#         self.y            = y
#         self.mode         = mode
#         self.n_folds      = n_folds
#         self.seed         = seed
#         self.reg          = reg 
#         self.modules      = modules
#         self.space_params = space_params
        
#     def process(self, space_steps, trials, algo, max_evals):
#         # fn = getattr(self, fn_name)    
#         try:
#             result = fmin(fn=self._struct_optimizer, space=space_steps, algo=algo, max_evals=max_evals, trials=trials)
#         except Exception as e:
#             return {'status': STATUS_FAIL,
#                     'exception': str(e)}
#         return result, trials

#     def _struct_optimizer(self, para):        
#         reg = Pipeline(construct_pipe(para, self.modules))      
#         space_filtered = filter_params(self.space_params, reg)
        
#         hpoptimizer = PipeHPOpt(
#             X       = self.X, 
#             y       = self.y, 
#             reg     = reg, 
#             mode    = self.mode, 
#             n_folds = self.n_folds, 
#             seed    = self.seed
#         )
#         lgb_opt, trials = hpoptimizer.process(space=space_filtered, trials=Trials(), algo=tpe.suggest, max_evals=10)
#         return {
#             'loss': np.min([r['loss'] for r in trials.results]), 
#             'params': para['set_params'], 
#             'status': STATUS_OK
#         } 